### Импорт необходимых библиотек

In [1]:
import requests  

from bs4 import BeautifulSoup
from fake_useragent import UserAgent

import pandas as pd
import tempfile, os, zipfile, io

from datetime import date, timedelta

Решение для создания списка дат для итерирования взято с [Stackoverflow](https://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python).

In [2]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

### Класс для сбора данных

In [3]:
class FinancialInfo:
    def __init__(self, start_date=None, end_date=None):
        ''' 
        start_date: Первый день, за который надо получить данные. Формат ввода: ДД.ММ.ГГГГ. По умолчанию: 01.01.2010
        end_date: Последний день, за который надо получить данные. Формат ввода: ДД.ММ.ГГГГ. По умолчанию: сегодня
        '''
        if start_date is None:
            self.start_date = date(2010, 1, 1)
            self.start_date_text = self.start_date.strftime("%d.%m.%Y")
        else:
            self.start_date_text = start_date
            sd = list(map(int, start_date.split('.')))
            self.start_date = date(sd[2], sd[1], sd[0])
        if end_date is None:
            self.end_date = date.today()
            self.end_date_text = self.end_date.strftime("%d.%m.%Y")
        else:
            self.end_date_text = end_date
            ed = list(map(int, end_date.split('.')))
            self.end_date = date(ed[2], ed[1], ed[0])
    
    def get_usdrub(self):
        '''
        Выгружает данные по курсу USDRUB из XML Банка России за даты, указанные в атрибутах объекта.
        '''
        usd = dict()
        st_usd = self.start_date.strftime("%d/%m/%Y")
        ed_usd = self.end_date.strftime("%d/%m/%Y")
        url_i = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={st_usd}&date_req2={ed_usd}&VAL_NM_RQ=R01235'
        response_i = requests.get(url_i, headers={'User-Agent': UserAgent().chrome}, timeout=5)
        tree_usd = BeautifulSoup(response_i.content, 'html.parser')
        for i in tree_usd.find_all('record'):
            usd[i.get('date')] = float(i.value.text.replace(',', '.'))
        return pd.DataFrame.from_dict(usd, orient='index', columns=['usdrub'])
        
    def get_cb_key_rate(self):
        '''
        Выгружает данные по ключевой ставке с сайта Банка России за даты, указанные в атрибутах объекта.
        '''
        key_policy_rate = dict()
        url_k = f'https://www.cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From={self.start_date_text}&UniDbQuery.To={self.end_date_text}'
        response_k = requests.get(url_k, headers={'User-Agent': UserAgent().chrome}, timeout=5)
        tree_key_policy_rate = BeautifulSoup(response_k.content, 'html.parser')
        key_rate = pd.read_html(str(tree_key_policy_rate.find('div', {'class': "table-wrapper"}).table), index_col='Дата')[0]
        key_rate.index.name = None
        key_rate['Ставка'] = key_rate['Ставка']/100
        key_rate.rename({'Ставка': 'cb_key_rate'}, axis=1, inplace=True)
        return key_rate
    
    def get_fed_rate(self):
        fed_rate = dict()

        url_fed = 'https://www.federalreserve.gov/datadownload/Output.aspx?rel=PRATES&filetype=zip'
        response_fed = requests.get(url_fed, headers={'User-Agent': UserAgent().chrome})

        # концепт распаковки взят с stackoverflow, но ссылка потерялась
        with response_fed, zipfile.ZipFile(io.BytesIO(response_fed.content)) as archive:
                data = archive.read('PRATES_data.xml')

        all_fed_data = BeautifulSoup(str(data), 'lxml').find(id='PRATES_POLICY_RATES').find_all('frb:obs')

        # НУЖНО ПОПРОБОВАТЬ ПЕРЕПИСАТЬ ФИЛЬТР ПО ДАТЕ
        for m in all_fed_data:
            date_lst = list(map(int, m.get('time_period').split('-')))
            date_dt = date(date_lst[0], date_lst[1], date_lst[2])
            if date_dt >= start_date and date_dt <= end_date:
                fed_rate[date_dt.strftime("%d.%m.%Y")] = float(m.get('obs_value'))       
        return pd.DataFrame.from_dict(fed_rate, orient='index', columns=['fed_rate'])
        
    def get_gold(self):
        '''
        Выгружает данные по цене золота из XML Банка России за даты, указанные в атрибутах объекта.
        '''
        gold_buy = dict()
        url_gold = f'http://www.cbr.ru/scripts/xml_metall.asp?date_req1={self.start_date_text}&date_req2={self.end_date_text}'
        response_gold = requests.get(url_gold, headers={'User-Agent': UserAgent().chrome})
        tree_gold = BeautifulSoup(response_gold.content, 'html.parser')
        for j in tree_gold.metall.find_all('record', code='1'):
            j_date = j.get('date')
            gold_buy[j_date] = float(j.buy.text.replace(',', '.'))
        return pd.DataFrame.from_dict(gold_buy, orient='index', columns=['gold'])
        
    def get_all(self):
        '''
        Выгружает данные по цене золота, курсу USDRUB, ключевой ставке Банка России за даты, указанные в атрибутах объекта.
        '''
        gold_and_usd = pd.merge(self.get_usdrub(),self.get_gold(),  
                                how='left', left_index=True, right_index=True)
        gu_cb = pd.merge(gold_and_usd, self.get_cb_key_rate(), 
                        how='left', left_index=True, right_index=True)
        return pd.merge(gu_cb, self.get_fed_rate(), 
                        how='left', left_index=True, right_index=True)